In [5]:
# LLM Prompt Template sample
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key="{YOUR_OPENAI_KEY}")

template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
prompt = template.format(adjective="famous", topic="place", city="seoul")
print(prompt)
print(llm.invoke(prompt))
print("\n")

prompt = template.format(adjective="popular", topic="reastaurant", city="San francisco")
print(prompt)
print(llm.invoke(prompt))
print("\n")

Tell me a famous place in seoul in 300 characters.
content='Gyeongbokgung Palace, the largest of the Five Grand Palaces in Seoul, is a stunning example of traditional Korean architecture. Built in 1395, it served as the main royal palace of the Joseon Dynasty. Visitors can explore its beautiful gardens, watch the changing of the guard ceremony, and visit the National Palace Museum of Korea nearby.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 20, 'total_tokens': 91, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None} id='run-0259b877-7e24-4a34-9594-74092db24531-0' usage_metadata={'input_tokens': 20, 'output_tokens': 71, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'

# Template Serialization

In [6]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
template.save("template.json")

In [7]:
!cat template.json

{
    "name": null,
    "input_variables": [
        "adjective",
        "city",
        "topic"
    ],
    "optional_variables": [],
    "output_parser": null,
    "partial_variables": {},
    "metadata": null,
    "tags": null,
    "template": "Tell me a {adjective} {topic} in {city} in 300 characters.",
    "template_format": "f-string",
    "validate_template": false,
    "_type": "prompt"
}

In [8]:
from langchain_core.prompts import load_prompt

loaded_template = load_prompt("template.json")
prompt = loaded_template.format(adjective="popular", topic="cafe", city="San francisco")
print(prompt)

Tell me a popular cafe in San francisco in 300 characters.


# Chat Prompt

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(model="gpt-4o-mini", api_key="{YOUR_OPENAI_KEY}")

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a tour guide."),
        ("human", "I'm planning to visit {Country}."),
        ("ai", "I'm tour guide for {Country}."),
        ("human", "{user_input}"),
    ]
)

prompt = chat_template.format_messages(
    Country="Korea", user_input="What is top5 best place to go there?"
)

print("Prompt :", prompt)
print("-" * 30)

aiMessage = chat.invoke(prompt)
print(aiMessage)

Prompt : [SystemMessage(content='You are a tour guide.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm planning to visit Korea.", additional_kwargs={}, response_metadata={}), AIMessage(content="I'm tour guide for Korea.", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is top5 best place to go there?', additional_kwargs={}, response_metadata={})]
------------------------------
content='Korea is a beautiful country with a rich history and vibrant culture. Here are the top five places you should consider visiting:\n\n1. **Seoul**: The capital city is a must-visit, blending modern skyscrapers with historic palaces. Don\'t miss Gyeongbokgung Palace, Bukchon Hanok Village, and the bustling streets of Myeongdong for shopping and street food. The N Seoul Tower offers stunning views of the city.\n\n2. **Busan**: Known for its beautiful beaches, Busan is a coastal city that offers a different vibe from Seoul. Visit Haeundae Beach, the picturesque

# Prompt Composition

In [11]:
from langchain_core.prompts import PromptTemplate

role_prompt = PromptTemplate.from_template("You are tour guide for {country}")
question_prompt = PromptTemplate.from_template(
    "Please tell me about {interest} in {country}"
)

full_prompt = role_prompt + question_prompt
full_prompt.format(country="Korea", interest="famous place to visit")

'You are tour guide for KoreaPlease tell me about famous place to visit in Korea'

# Pipeline prompt template

In [15]:
from langchain_core.prompts import PipelinePromptTemplate
from langchain_core.prompts import PromptTemplate

full_template = """{role}

{question}

Please do not reply with anything other than information related to travel to {country} and reply “I cannot answer.”
"""

full_prompt = PromptTemplate.from_template(full_template)
role_prompt = PromptTemplate.from_template("You are tour guide for {country}")
question_prompt = PromptTemplate.from_template(
    "Please tell me about {interest} in {country}"
)

# composition
input_prompts = [("role", role_prompt), ("question", question_prompt)]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

prompt_text = pipeline_prompt.format(country="Korea", interest="famous place to visit")

print(prompt_text)

You are tour guide for Korea

Please tell me about famous place to visit in Korea

Please do not reply with anything other than information related to travel to Korea and reply “I cannot answer.”



# Prompt Serialization

In [14]:
pipeline_prompt.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'pipeline', 'PipelinePromptTemplate'],
 'kwargs': {'input_variables': ['country', 'interest'],
  'final_prompt': PromptTemplate(input_variables=['country', 'question', 'role'], input_types={}, partial_variables={}, template='{role}\n\n{question}\n\nPlease do not reply with anything other than information related to travel to {country} and reply “I cannot answer.”\n'),
  'pipeline_prompts': [('role',
    PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='You are tour guide for {country}')),
   ('question',
    PromptTemplate(input_variables=['country', 'interest'], input_types={}, partial_variables={}, template='Please tell me about {interest} in {country}'))]},
 'name': 'PipelinePromptTemplate'}

# Partial prompt templates

In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="What is famous {topic} in {city}?", input_variables=["topic", "city"]
)
partial_prompt = prompt.partial(city="Seoul")
print(partial_prompt.format(topic="food"))

What is famous food in Seoul?


In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="What is famous {topic} in {city}?",
    input_variables=["topic"],
    partial_variables={"city": "seoul"},
)
partial_prompt = prompt.partial(city="Seoul")
print(partial_prompt.format(topic="food"))

What is famous food in Seoul?
